<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>7. Proyecto de regresión</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Topología de linea base](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de la toplogía](#section3)
    * [3.1. Evaluar una topología más profunda](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, trabajaremos cómo desarrollar y evaluar modelos de redes neuronales para un problema de regresión:
* Cómo crear un modelo de red neuronal para un problema de regresión.
* Cómo utilizar scikit-learn con Keras para evaluar modelos mediante validación cruzada.
* Cómo realizar la preparación de datos para mejorar la habilidad.
* Cómo Optimizar la topología de red de modelos.

In [10]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Topología de linea base</font>

En esta sección crearemos un modelo de red neuronal de referencia para el problema de regresión. Comencemos importando todas las funciones y objetos que necesitaremos para este tutorial.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Boston House Price](http://lib.stat.cmu.edu/datasets/boston)

In [5]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# load dataset
r = "C:/Users/Maria Luisa/OneDrive - ECON Tech/Frisa_Calculo_Hidrogeno/Modelo Hidrogeno Ago-Dic_15-01.xlsx"

df1 = pd.read_excel(r)
dataset = df1.values
# split into input (X) and output (Y) variables
#X = dataset[:, 0:13]
#y = dataset[:,13]
df1

,Colada,Orden,Grado,Perfil,FechaProducción,Muestra,Inoxidable,Duración,ModeloHidrógeno,HidrógenoPPM,...,VDStartTemp,VDDuration TOTAL,VDPowerOn,VDPowerOff,VDEvacuationDuration,VDDeepVacuunDuration,VDPressureMin,VDArgonTotal,VDNitrogenTotal,VDEndTemp
0,1207532,60052459,42CRMO4 LIEBHERR,"24""R",2020-08-01,VD03,no,20,0.98,1.12,...,1652,60,27,33,7,20,0.67,4.44,0.0,1563
1,1207533,60052354,4140,"16""R",2020-08-01,VD02,no,19,1.17,0.81,...,1668,54,25,29,6,19,0.58,3.91,0.0,1581
2,1207534,60052387,A105,"52""P",2020-08-01,VD03,no,22,0.78,0.65,...,1684,61,29,32,7,22,0.78,10.69,0.0,1577
3,1207535,60052524,A105,"39""R",2020-08-01,VD02,no,16,1.10,1.10,...,1669,54,22,32,6,16,0.71,3.55,0.0,1586
4,1207536,60052371,1020,"20""R",2020-08-01,VD02,no,20,1.10,1.10,...,1690,52,26,26,6,20,0.68,3.53,0.0,1598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,1208354,60057797,1524 CAT,"20""R",2020-12-22,VD02,no,24,0.72,0.99,...,1703,56,29,27,5,24,0.58,9.86,0.0,1596
823,1208355,60058173,A871 TYPE 1,"31""R",2020-12-22,VD03,no,24,0.71,0.71,...,1685,60,29,31,5,24,0.64,10.78,0.0,1587
824,1208356,60058289,1080,"24""R",2020-12-22,VD03,no,22,0.71,0.67,...,1643,63,28,35,6,22,0.58,8.41,0.0,1551
825,1208357,60058295,E8630M,"31""R",2020-12-22,VD02,no,24,0.72,0.82,...,1680,54,29,25,5,24,0.56,5.81,0.0,1580


# Data cleaning

In [58]:
df1.drop(index = (df1[df1['Prevaciado']<0].index.to_list()), axis=0 , inplace=True)
df1 = df1[df1['HidrógenoPPM'] <3]
df1 = df1[df1['Tapping'] <80000]

arreglo_Nelson = ['HidrógenoPPM', 'Duración', 'Prevaciado','DurationDeepVacuum_1mbar',  'Presión_Promedio_de_Vacío_Profundo_mbar', 'Consumo_promedio_de_Ar_durante_VD_Nm3',
                  'C', 'Si', 'Mn', 'Ni', 'Al', 'P', 'S','Cr', 'OffGasCO2', 'OffGasCO','OffGasH', 'Factor_Kf_Temp','VDPowerOn', 'VDEndTemp','Tapping', 'ModeloHidrógeno' ]
df2 = df1.loc[:, arreglo_Nelson]
df2 = df2.dropna()
modelofrisa = df2.loc[:,'ModeloHidrógeno']
df2 = df2.drop(columns = ['ModeloHidrógeno'], axis=1)
data_dict = { 'Factor' : [ 0.06,-0.0014,0.011,0.008,0.023,0.013,-0.0022,-0.0016],
         'Análisis químico del acero': [0.150,1.100,0.007,0.004,0.200,0.017,18.000,0.200]}
data = pd.DataFrame.from_dict(data_dict).rename(index=dict(zip(range(0,8 ), ['C', 'Mn', 'P', 'S', 'Si', 'Al', 'Cr', 'Ni'])))
data_kf = df2
data_kf['sum_f_X_elementos'] = (data_kf['C'] * data.loc['C','Factor'] + data_kf['Mn'] * data.loc['Mn','Factor'] +
                                data_kf['P'] * data.loc['P','Factor'] +data_kf['S'] * data.loc['S','Factor'] +
                                data_kf['Si'] * data.loc['Si','Factor'] + data_kf['Al'] * data.loc['Al','Factor']+
                                data_kf['Cr'] * data.loc['Cr','Factor'] + data_kf['Ni'] * data.loc['Ni','Factor']
                                )
data_kf['Fh'] =  10**(data_kf['sum_f_X_elementos'])
log_kht = 0.923 - (1900/ (df2['Factor_Kf_Temp'] + 273) )
data_kf['khT'] = 10**(log_kht)
data_kf['K_F'] =  data_kf['khT'] /data_kf['Fh']
data_kf = data_kf.astype(float)


In [59]:
X = data_kf.iloc[:,1:]
y = data_kf.iloc[:,0].values
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 379 to 816
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Duración                                 421 non-null    float64
 1   Prevaciado                               421 non-null    float64
 2   DurationDeepVacuum_1mbar                 421 non-null    float64
 3   Presión_Promedio_de_Vacío_Profundo_mbar  421 non-null    float64
 4   Consumo_promedio_de_Ar_durante_VD_Nm3    421 non-null    float64
 5   C                                        421 non-null    float64
 6   Si                                       421 non-null    float64
 7   Mn                                       421 non-null    float64
 8   Ni                                       421 non-null    float64
 9   Al                                       421 non-null    float64
 10  P                                        421 non

In [43]:
X.shape

(421, 24)

# Comenzamos con la construccion del modelo

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Tiene una sola capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13). 
2. La red utiliza la función de activación ReLu para la capa oculta.
    * Al ser problema de regresión no tiene función de activación en la capa de salida
3. Utiliza el algoritmo de optimización ADAM y se optimiza una función de pérdida de error cuadrático medio. 
4. El objeto Wrapper para regresión se llama `KerasRegressor`. 
5. Evaluamos este modelo de línea de base con 10-fold.

In [104]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(24, input_dim =24, activation = 'relu'))
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(60,   activation = 'relu'))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('NN', KerasRegressor(build_fn=baseline_model, epochs=100,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv =kfold)
print(results)
print('Linea base: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

[-0.05946355 -0.13706905 -0.1277208  -0.0530308  -0.28166658 -0.11133144
 -0.11395792 -0.08889373 -0.09875599 -0.40844962]
Linea base: -0.15 (0.11)MSE 


In [105]:
y_pred = baseline_model().predict(X)
y_s = pd.concat([pd.DataFrame(y_pred), pd.DataFrame(y)], axis=1)
y_s


,0,0
0,-1423.737549,0.74
1,-1423.686035,1.08
2,-1505.231567,0.86
3,-1444.113037,0.50
4,-1488.316650,1.25
...,...,...
416,-1507.253052,2.04
417,-1391.907471,0.79
418,-1486.249512,0.63
419,-1390.820801,0.77


In [100]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(24, input_dim =24, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('NN', KerasRegressor(build_fn=baseline_model, epochs=100,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv =kfold)
print(results)
print('Linea base: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

[-0.15125273 -0.12133182 -0.13369067 -0.05187222 -0.27408642 -0.12352256
 -0.10851767 -0.08909117 -0.10251088 -0.10390619]
Linea base: -0.13 (0.06)MSE 


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

In [62]:
print('Linea base: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

Linea base: -351.78 (475.94)MSE 


<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Podemos utilizar el marco de trabajo [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para realizar la **estandarización** durante el proceso de evaluación del modelo, dentro de cada fold. 

In [63]:
# Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate model with standardized dataset
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('NN', KerasRegressor(build_fn=baseline_model, epochs=100,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('Estandarizado: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

Estandarizado: -643.62 (990.78)MSE 


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
**Trabajo a realizar:** Una extensión adicional de esta sección sería aplicar de manera similar un cambio de escala a la variable de salida, como normalizarla al rango de 0 a 1 y usar una función de activación Sigmoide o similar en la capa de salida para reducir las predicciones de salida al mismo rango.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de la toplogía</font>

En esta sección evaluaremos dos topologías (más profunda y amplia) para mejorar aún más el rendimiento del modelo.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más profunda</font>

Una forma de mejorar el rendimiento de una red neuronal es agregar más capas para hacerla más profunda. En este caso con aproximadamente la mitad del número de neuronas. Nuestra topología de red ahora se ve así:
```
    13 entradas -> [13 -> 6] -> 1 salida
```
Podemos evaluar esta topología de red de la misma manera que anteriormente, al mismo tiempo que usamos la estandarización que demostró un mejor rendimiento.

In [64]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim =13, activation = 'relu'))
    model.add(Dense(6, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('DeepNN', KerasRegressor(build_fn=larger_model, epochs=100,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('DeepNN: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

DeepNN: -532.57 (671.91)MSE 


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Otro enfoque para aumentar la capacidad de representación del modelo es crear una red más amplia. Aquí, hemos aumentado el número de neuronas en la capa oculta en de 13 a 20. La topología de nuestra red más amplia se puede resumir de la siguiente manera:

```
    13 entradas -> [20] -> 1 salida
```

In [65]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim =13, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('Deep wider NN', KerasRegressor(build_fn=wider_model, epochs=50,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('Deep wider NN: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

Deep wider NN: -1304.58 (1671.54)MSE 


La construcción del modelo ve una caída adicional en el error a aproximadamente 22 mil dólares cuadrados. Este no es un mal resultado para este problema.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>